In [ ]:
from pathlib import Path
import pandas as pd

INPUT_PATH = Path(r"C:\Users\w322800\Documents\gh\bcsb-prod\Reports\Retail\Accubranch\output\transaction.parquet")
df = pd.read_parquet(INPUT_PATH)

In [ ]:
df

In [ ]:
df = df[df['Account Type'] != "Other"].copy()

In [ ]:
df['Type of Teller'].unique()

In [ ]:
df

In [ ]:
# df = df[df['Branch of Transaction'].notna()].copy()

In [ ]:
df = df.reset_index()

In [ ]:
df.groupby('Type of Teller')['index'].nunique().sort_values(ascending=False)

In [ ]:
df

In [ ]:
df.groupby('Branch of Transaction')['index'].nunique().sort_values(ascending=False)

In [ ]:
df

In [ ]:

import cdutils.database.connect # type: ignore
import cdutils.deduplication # type: ignore
from sqlalchemy import text # type: ignore
from datetime import datetime, timedelta
from typing import Optional

def fetch_transactions_window(
    start_date: Optional[datetime] = None,
    end_date: Optional[datetime] = None,
    window_size_days: int = 365
) -> list:
    """
    Fetch transactions within a specified date window from the database.
    :param start_date: The start date for the transaction window. If None, defaults to today minus window size.
    :param end_date: The end date for the transaction window. If None, defaults to today.
    :param window_size_days: The number of days to look back from the end date.
    :return: A list of transactions within the specified date range.
    """
    
    if start_date is None:
        start_date = datetime.now() - timedelta(days=window_size_days)
    
    if end_date is None:
        end_date = datetime.now()

    query = text(f"""
        SELECT 
            * 
        FROM
            COCCDM.WH_RTXN a
        WHERE 
            (a.RUNDATE >= TO_DATE('{start_date.strftime('%Y-%m-%d')}', 'YYYY-MM-DD')) 
            AND (a.RUNDATE <= TO_DATE('{end_date.strftime('%Y-%m-%d')}', 'YYYY-MM-DD'))
            AND (a.CURRRTXNSTATCD = 'C')
        ORDER BY RUNDATE DESC
        FETCH FIRST 100000 ROWS ONLY
    """)

    # Add WH_ORG query for branch names
    wh_org_query = text("""
        SELECT 
            *
        FROM
            OSIBANK.WH_ORG
    """)

    queries = [
        {'key': 'query','sql': query,'engine':2},
        {'key': 'wh_org','sql': wh_org_query,'engine':1}
    ]
    data = cdutils.database.connect.retrieve_data(queries)
    
    # Deduplicate WH_ORG data
    if 'wh_org' in data:
        dedupe_list = [{'df': data['wh_org'], 'field': 'orgnbr'}]
        data['wh_org'] = cdutils.deduplication.dedupe(dedupe_list)
    
    return data



In [ ]:
raw_data = fetch_transactions_window()

In [ ]:
raw_data.keys()

In [ ]:
rtxn = raw_data['query'].copy()

In [ ]:
rtxn